In [ ]:
import DataHelpers
import AugmentationHelpers
import InferenceHelpers
import EnsembleHelpers
import TrainingHelpers
import os
import torch
import pandas as pd
#Directory containing the training images of DFUC 2021
TRAIN_DATA_DIR = "./data/train/images/"
#.csv file containing the labels of the training dataset
TRAIN_LABELS_FILE=os.path.abspath("./data/train/train.csv")
#Directory to save trained models
MODEL_DIR = "./Ensemble_Models/"


In [ ]:
#File with model parameters
MODEL_PARAMETERS_FILE="./Hyperparameters_Ensemble_Model_1+2+3+4.csv"


In [ ]:
model_parameters=pd.read_csv(MODEL_PARAMETERS_FILE,sep=";")
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
for indexi, settings in model_parameters.iterrows():
    MODEL_DIR_SUB=MODEL_DIR+"/ID"+str(settings["ID"])+"/"
    if not os.path.exists(MODEL_DIR_SUB):
        os.mkdir(MODEL_DIR_SUB)
        MODEL_NAME=settings["Model"]
        PRETRAINED=settings["Pretrained"]
        learning_rate_warm_up=settings["Learning rate warm up"]
        learning_rate=settings["Learning rate training"]
        use_scheduler=settings["Scheduler"]
        BATCH_SIZE = settings["Batch size"]
        IMAGE_SIZE = settings["Image size"]
        augmentation=settings["Augmentations"]
        NUM_EPOCHS_WARMUP=settings["Num epochs warm up"]
        NUM_EPOCHS=settings["Num epochs"]
        use_oversampling=settings["Oversampling"]
        optimizer_type=settings["Optimizer"]
    #set model name, that corresponds to timm model names
        drop_rate=settings["Drop rate"]
        multi_GPU=settings["Multi GPU"]

        device = ("cuda" if torch.cuda.is_available() else "cpu")
        torch.manual_seed(103)
        trainingDataset=DataHelpers.prepare_Training_Dataset(TRAIN_LABELS_FILE,TRAIN_DATA_DIR)
        NUM_CLASSES=DataHelpers.get_number_of_classes_in_dataset(trainingDataset)
        DFUCDataset=DataHelpers.initialize_DFUC_Dataloader()
        train_augmentations=AugmentationHelpers.load_training_augmentations(IMAGE_SIZE,augmentation)
        train_loader=DataHelpers.load_dataset(trainingDataset,DFUCDataset,train_augmentations,True,BATCH_SIZE,use_oversampling)
        valid_loader=None
        num_training_samples=trainingDataset.shape[0]
        num_validation_samples=0
        model=TrainingHelpers.iterate_epochs(train_loader,valid_loader,MODEL_NAME,optimizer_type,drop_rate,num_training_samples,num_validation_samples,NUM_EPOCHS+NUM_EPOCHS_WARMUP,MODEL_DIR_SUB,device, learning_rate,learning_rate_warm_up,use_scheduler,NUM_EPOCHS_WARMUP,PRETRAINED,NUM_CLASSES,multi_GPU)